## Deploy endpoint with the trained data

After model training, we will deploy an endpoint with trained model. But before doing that it is required that zips the deployment codes and model file into a zip file and upload the file to S3.

In [ ]:
import sagemaker as sm
print('SageMaker:{}'.format(sm.__version__))
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer

In [ ]:
class JSONPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(JSONPredictor, self).__init__(endpoint_name, sagemaker_session, json_serializer, json_deserializer)

In [ ]:
role = get_execution_role()

In [ ]:
env = {
    'SAGEMAKER_MODEL_SERVER_WORKERS': '1'
}

fd_sl_model = PyTorchModel(model_data='s3://fraud-detection-solution/model/fd_sl_model.tar.gz', 
                           role=role,
                           entry_point='fd_sl_deployment_entry_point.py',
                           framework_version='1.4.0',
                           py_version='py3',
                           predictor_cls=JSONPredictor,
                           env=env)

In [ ]:
fd_sl_predictor = fd_sl_model.deploy(instance_type='ml.c4.4xlarge', 
                                     initial_instance_count=1,
#                                      endpoint_name='FraudDetectionSolutionTestV01')
                                    )

In [ ]:
fd_sl_predictor.delete_endpoint()
fd_sl_model.delete_model()